In [121]:
import pandas as pd
import numpy as np
#from autoviz.AutoViz_Class import AutoViz_Class
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [122]:
appli = pd.read_csv('application_data.csv')
prev = pd.read_csv("previous_application.csv")
col = pd.read_csv('columns_description.csv')

In [123]:
appli = appli.iloc[0:50000]
prev = prev.iloc[0:50000]
#appli = appli.iloc[:, : 42]
#prev = prev.iloc[:, : 42]

In [124]:
prev.shape, appli.shape

((50000, 37), (50000, 122))

In [125]:
miss_application_data = pd.DataFrame((prev.isnull().sum())*100/prev.shape[0]).reset_index()
miss_application_data.head()

,index,0
0,SK_ID_PREV,0.000
1,SK_ID_CURR,0.000
2,NAME_CONTRACT_TYPE,0.000
3,AMT_ANNUITY,21.184
4,AMT_APPLICATION,0.000


## deux options, laquelle tu pref ?

1 - delete comme en dessous

In [126]:
prev = prev.loc[:, prev.isnull().mean() < 0.30] #supprime les cols avec 30% de valeurs manquantes
prev.shape

(50000, 26)

2 - remplacer par valeurs médianes ? 

In [160]:
vec = miss_application_data.loc[miss_application_data[0]>=30]["index"]
vec

6              AMT_DOWN_PAYMENT
12            RATE_DOWN_PAYMENT
13        RATE_INTEREST_PRIMARY
14     RATE_INTEREST_PRIVILEGED
20              NAME_TYPE_SUITE
31           DAYS_FIRST_DRAWING
32               DAYS_FIRST_DUE
33    DAYS_LAST_DUE_1ST_VERSION
34                DAYS_LAST_DUE
35             DAYS_TERMINATION
36    NFLAG_INSURED_ON_APPROVAL
Name: index, dtype: object

In [128]:
# a finir
'''
for i in vec:
    prev.loc[prev[i]==prev[i].median()]
'''

'\nfor i in vec:\n    prev.loc[prev[i]==prev[i].median()]\n'

In [129]:
appli = appli.loc[:, appli.isnull().mean() < 0.30] #supprime les cols avec 30% de valeurs manquantes
appli.shape

(50000, 72)

### rencodage selon type

In [130]:
cate = [i for i in prev.select_dtypes(include = np.object).columns if i not in ["type"] ]
num = [i for i in prev.select_dtypes(include = np.number).columns if i not in ['SK_ID_CURR'] + [ 'TARGET']]

C:\Users\Adrien\AppData\Local\Temp/ipykernel_8484/1768624195.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cate = [i for i in prev.select_dtypes(include = np.object).columns if i not in ["type"] ]


In [145]:
application_data_x = appli[[x for x in appli.columns]]
previous_application_x = prev[[x for x in prev.columns]]
application_data_x["type"] = "application_data"
previous_application_x["type"] = "previous_application"
data = pd.concat([application_data_x,previous_application_x],axis=0) 
data = data[data['TARGET'].notna()]

## a partir d'ici, plus de nan values pour les perfs du modèle

In [154]:
from sklearn.model_selection import train_test_split

y = data.TARGET
X = data.drop(['TARGET'],axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(
                                                    X, y, train_size=0.8, test_size=0.2, random_state=0)

cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)


low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

C:\Users\Adrien\anaconda3\envs\machlearn\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [155]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

### implémentation possible pour un DF qui se mettrait à jour 

In [ ]:
from zlib import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

data_withid = data.reset_index() # adds an `index` column
train_set, test_set = split_train_test_by_id(data_withid, 0.2, "index")

In [149]:
X_train

(40000, 90)

In [157]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [158]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from One-Hot Encoding:") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from One-Hot Encoding:
0.156579


ici, model pour selectionner les features importantes

In [159]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=1)

model.fit(OH_X_train,y_train)

DecisionTreeRegressor(random_state=1)

ici, utilisateur rentre les infos pr les features importantes et on lui donne un score / ou 